# Logistic Regression

- similar to linear regression in that logistic regression predicts a target based on weights assigned to each feature
- sigmoid function is used to constrain the predicted probability to between 0 and 1
- sigmoid function $f(x) = \frac{1}{1+e^{-x}}$
- as x approaches infinity, f(x) gets closer to 1
- as x approaches negative infinity, f(x) gets closer to 0

Why use logistic regression for classification problems?
- features can be dichotomous, categorical or numerical
- features does not need to be normally distributed
- the target variable can have more than two possible values, and can be nominal or ordinal
- relationship between features and the target is not assumed to be linear

## Binary Classification with Logistic Regression

### Goal

We will build a model to predict if an individual will have heart disease based on personal characteristics such as smoking and alcohol drinking habits; health features, including BMP, asthma, diabetes, and skin cancer; and age.

### Importing dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics as skmet
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFECV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from data.load import load_healthinfosample

# Custom dependencies
from preprocfunc import MakeOrdinal, OutlierTrans, ReplaceVals

### Load data

In [2]:
healthinfo = load_healthinfosample()
healthinfo.head(2).T

personid,299391,252786
heartdisease,Yes,No
bmi,28.48,25.24
smoking,Yes,Yes
alcoholdrinkingheavy,No,No
stroke,No,No
physicalhealthbaddays,7,0
mentalhealthbaddays,0,2
walkingdifficult,No,No
gender,Male,Female
agecategory,70-74,65-69


^ The `agecategory` feature is a character data that represents the age interval. We will need to convert that feature into numeric.

```
agecategory	70-74	65-69
```

### Check for missing values

In [3]:
healthinfo.shape

(30000, 18)

In [4]:
healthinfo.isnull().sum()

heartdisease             0
bmi                      0
smoking                  0
alcoholdrinkingheavy     0
stroke                   0
physicalhealthbaddays    0
mentalhealthbaddays      0
walkingdifficult         0
gender                   0
agecategory              0
ethnicity                0
diabetic                 0
physicalactivity         0
genhealth                0
sleeptimenightly         0
asthma                   0
kidneydisease            0
skincancer               0
dtype: int64

^ We don't have any missing values. 

### Target column

The target column is `heartdisease`. We need to transform it into a `0` and `1` variable.

In [5]:
healthinfo.heartdisease.value_counts()

No     27467
Yes     2533
Name: heartdisease, dtype: int64

In [6]:
healthinfo.heartdisease.value_counts(normalize=True)

No     0.915567
Yes    0.084433
Name: heartdisease, dtype: float64

^ Less than 10% of our observations have heartdisease. Our target's values are quite imbalanced.

In [7]:
healthinfo["heartdisease"] = np.where(healthinfo.heartdisease == "No", 0, 1).astype(int)

In [8]:
healthinfo.heartdisease.value_counts()

0    27467
1     2533
Name: heartdisease, dtype: int64

### Preprocessing

- scaling `numeric` features
- one-hot encoding `categorical` features
- convert to `ordinal` features

In [9]:
num_cols = ["bmi", "physicalhealthbaddays", "mentalhealthbaddays", "sleeptimenightly"]
binary_cols = [
    "smoking",
    "alcoholdrinkingheavy",
    "stroke",
    "walkingdifficult",
    "physicalactivity",
    "asthma",
    "kidneydisease",
    "skincancer",
]

cat_cols = ["gender", "ethnicity"]
spec_cols1 = ["agecategory"]
spec_cols2 = ["genhealth"]
spec_cols3 = ["diabetic"]

In [10]:
rep_dict = {
    "genhealth": {"Poor": 0, "Fair": 1, "Good": 2, "Very good": 3, "Excellent": 4},
    "diabetic": {
        "No": 0,
        "No, borderline diabetes": 0,
        "Yes": 1,
        "Yes (during pregnancy)": 1,
    },
}

### EDA
- explore binary features
- explore categorical features
- explore numeric features

In [11]:
healthinfo[binary_cols].apply(pd.value_counts, normalize=True).T

,No,Yes
smoking,0.584300,0.415700
alcoholdrinkingheavy,0.933133,0.066867
stroke,0.962200,0.037800
walkingdifficult,0.862467,0.137533
physicalactivity,0.227267,0.772733
asthma,0.865500,0.134500
kidneydisease,0.962500,0.037500
skincancer,0.906233,0.093767


^ how to interpret? A large percentage of the individuals (42%) report that they have been smokers. 14% report that they have difficulty walking.

Template:

```
n% report that <cause>
```

In [12]:
for col in cat_cols + ["genhealth", "diabetic"]:
    print(
        col,
        "------------------------------",
        healthinfo[col].value_counts(normalize=True).sort_index(),
        sep="\n",
        end="\n\n",
    )

gender
------------------------------
Female    0.5229
Male      0.4771
Name: gender, dtype: float64

ethnicity
------------------------------
American Indian/Alaskan Native    0.017000
Asian                             0.026333
Black                             0.070667
Hispanic                          0.085733
Other                             0.033533
White                             0.766733
Name: ethnicity, dtype: float64

genhealth
------------------------------
Excellent    0.207200
Fair         0.110733
Good         0.291533
Poor         0.035400
Very good    0.355133
Name: genhealth, dtype: float64

diabetic
------------------------------
No                         0.844767
No, borderline diabetes    0.022367
Yes                        0.125100
Yes (during pregnancy)     0.007767
Name: diabetic, dtype: float64



^ There are nearly equal numbers of men and women.

Most people report excellent or very good health.

In [13]:
healthinfo[num_cols].agg(["count", "min", "median", "max"]).T

,count,min,median,max
bmi,30000.0,12.02,27.34,91.55
physicalhealthbaddays,30000.0,0.00,0.00,30.00
mentalhealthbaddays,30000.0,0.00,0.00,30.00
sleeptimenightly,30000.0,1.00,7.00,24.00


- we need to do scaling
- we need to do some encoding of the categorical features
- we need to remove extreme values for some numerical features. A `sleeptimenightly` value of 24 seems unlikely

### Create train and test data

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    healthinfo[
        num_cols + binary_cols + cat_cols + spec_cols1 + spec_cols2 + spec_cols3
    ],
    healthinfo["heartdisease"],
    test_size=0.2,
    random_state=0,
)

### Column Transformation

- categorical features - one-hot encoding
- numeric features - remove extreme values, impute median
- convert `agecategory` features into ordinal values using `MakeOrdinal`
- replace `genhealth` and `diabetic` features using the `ReplaceVals`

In [15]:
ohe = OneHotEncoder(drop="first", sparse=False)

standtrans = make_pipeline(
    OutlierTrans(3), SimpleImputer(strategy="median"), StandardScaler()
)
spectrans1 = make_pipeline(MakeOrdinal(), StandardScaler())
spectrans2 = make_pipeline(ReplaceVals(rep_dict), StandardScaler())
spectrans3 = make_pipeline(ReplaceVals(rep_dict))
bintrans = make_pipeline(ohe)
cattrans = make_pipeline(ohe)
coltrans = ColumnTransformer(
    transformers=[
        ("stand", standtrans, num_cols),
        ("spec1", spectrans1, spec_cols1),
        ("spec2", spectrans2, spec_cols2),
        ("spec3", spectrans3, spec_cols3),
        ("bin", bintrans, binary_cols),
        ("cat", cattrans, cat_cols),
    ]
)

### Setup pipeline

- setup a logistic regression and instantiate a stratified k-fold object
- recall that the recursive feature elimination needs an estimator
- remember that our dataset is imbalanced
- we use stratified k-fold to get approximately the same target value distribution in each fold
- we create another logistic regresion instance for our model
- set the `class_weight` to `balanced` 
- this should improve the model's ability to deal with the class imbalance

In [16]:
lrsel = LogisticRegression(random_state=1, max_iter=1000)
kf = StratifiedKFold(n_splits=5, shuffle=True)
rfecv = RFECV(estimator=lrsel, cv=kf)
lr = LogisticRegression(random_state=1, class_weight="balanced", max_iter=1000)

pipe1 = make_pipeline(coltrans, rfecv, lr)
pipe1.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('stand',
                                                  Pipeline(steps=[('outliertrans',
                                                                   OutlierTrans(threshold=3)),
                                                                  ('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['bmi',
                                                   'physicalhealthbaddays',
                                                   'mentalhealthbaddays',
                                                   'sleeptimenightly']),
                                                 ('spec1',
                                                  Pipeline(steps=[('makeordinal',
                                                                   MakeOrdina...
                                                   'skincancer']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  ['gender', 'ethnicity'])])),
                ('rfecv',
                 RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
                       estimator=LogisticRegression(max_iter=1000,
                                                    random_state=1))),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    random_state=1))])

We need to do a little work to recover the column names from the pipeline after the fit. 

We should only get the column names for the binary and categorical features after the encoding. The names of the numerical features remained unchanged.

In [26]:
new_binary_cols = (
    pipe1.named_steps["columntransformer"]
    .named_transformers_["bin"]
    .named_steps["onehotencoder"]
    .get_feature_names_out()
)
new_binary_cols

array(['smoking_Yes', 'alcoholdrinkingheavy_Yes', 'stroke_Yes',
       'walkingdifficult_Yes', 'physicalactivity_Yes', 'asthma_Yes',
       'kidneydisease_Yes', 'skincancer_Yes'], dtype=object)

In [27]:
new_cat_cols = (
    pipe1.named_steps["columntransformer"]
    .named_transformers_["cat"]
    .named_steps["onehotencoder"]
    .get_feature_names_out()
)
new_cat_cols

array(['gender_Male', 'ethnicity_Asian', 'ethnicity_Black',
       'ethnicity_Hispanic', 'ethnicity_Other', 'ethnicity_White'],
      dtype=object)

In [29]:
new_cols = np.concatenate(
    (
        np.array(num_cols + spec_cols1 + spec_cols2 + spec_cols3),
        new_binary_cols,
        new_cat_cols,
    )
)
new_cols

array(['bmi', 'physicalhealthbaddays', 'mentalhealthbaddays',
       'sleeptimenightly', 'agecategory', 'genhealth', 'diabetic',
       'smoking_Yes', 'alcoholdrinkingheavy_Yes', 'stroke_Yes',
       'walkingdifficult_Yes', 'physicalactivity_Yes', 'asthma_Yes',
       'kidneydisease_Yes', 'skincancer_Yes', 'gender_Male',
       'ethnicity_Asian', 'ethnicity_Black', 'ethnicity_Hispanic',
       'ethnicity_Other', 'ethnicity_White'], dtype=object)

### Recursive Feature Elimination

We can use the `ranking_` attribute of the `rfecv` object to get the ranking of each feature. Those with `1` for ranking will be selected for our model.

If we use the `get_support` method or the `support_` attribute of the `rfecv` object, we get just those features that will be used in our model.

In [31]:
rankinglabs = np.column_stack((pipe1.named_steps["rfecv"].ranking_, new_cols))
pd.DataFrame(rankinglabs, columns=["rank", "feature"]).sort_values(
    ["rank", "feature"]
).set_index("rank")

,feature
rank,
1,agecategory
1,alcoholdrinkingheavy_Yes
1,asthma_Yes
1,diabetic
1,ethnicity_Asian
1,gender_Male
1,genhealth
1,kidneydisease_Yes
1,smoking_Yes


In [32]:
pipe1.named_steps["rfecv"].support_

array([False, False, False, False,  True,  True,  True,  True,  True,
        True, False, False,  True,  True, False,  True,  True, False,
       False, False, False])

In [41]:
new_cols[pipe1.named_steps["rfecv"].get_support()]

array(['agecategory', 'genhealth', 'diabetic', 'smoking_Yes',
       'alcoholdrinkingheavy_Yes', 'stroke_Yes', 'asthma_Yes',
       'kidneydisease_Yes', 'gender_Male', 'ethnicity_Asian'],
      dtype=object)

### Logistic Regression coefficients

We can get the _odds ratio_ from the coefficients from the logistic regression. Recall that the odds ratio is the exponentiated coefficient. There are 10 coefficients because we have 10 features.

In [43]:
oddsratios = np.exp(pipe1.named_steps["logisticregression"].coef_)

oddsratios.shape

(1, 10)

In [47]:
sel_cols = new_cols[pipe1.named_steps["rfecv"].get_support()]
oddswithlabs = np.column_stack((oddsratios.ravel(), sel_cols))

pd.DataFrame(oddswithlabs, columns=["odds", "feature"]).sort_values(
    ["odds"], ascending=False
).set_index("odds")

,feature
odds,
3.132956,stroke_Yes
2.950606,agecategory
2.095302,gender_Male
2.038024,kidneydisease_Yes
1.780634,diabetic
1.575736,smoking_Yes
1.538203,asthma_Yes
0.615857,alcoholdrinkingheavy_Yes
0.602362,ethnicity_Asian


^ If the individual had stroke, they had three times the odds of having heart disease. The odds of having heart disease increase by 2.95 times for each increase in age category.

On the other hand, the odds of having heart disease decline by half (55%) for every increase in general health. Surprisingly, heavy alcohol drinking is associated with lower odds of heart disease.

### Evaluating a logistic regression model